In [1]:
# Load or install the program, %%capture supresses print statements
!pip install DicomRTTool --upgrade
!pip install flywheel-sdk 
!pip install antspyx
!pip install --default-timeout=200 future # ensure that no timeouts occur when connecting to flywheel

You should consider upgrading via the '/Users/cxl037/PycharmProjects/pythonProject1/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/cxl037/PycharmProjects/pythonProject1/venv/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/cxl037/PycharmProjects/pythonProject1/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/cxl037/PycharmProjects/pythonProject1/venv/bin/python -m pip install --upgrade pip' command.


In [10]:
# importing neccessary libraries 

from DicomRTTool.ReaderWriter import DicomReaderWriter

# file management 
import os 
import zipfile
import gzip
import shutil
from six.moves import urllib
import flywheel

# array manipulation and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# medical image manipulation 
import SimpleITK as sitk

# image registration
import ants

import pandas

In [29]:
root = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/'

In [58]:
# initialize flywheel variables
fw = flywheel.Client('flywheel.uwhealth.org:0tfO3O6KmTcoy0fVxM')
FW_GROUP = 'baschnagelgroup'
FW_PROJECT = 'SRS Necrosis Project'
image_path = root + 'Image_Data'
if not os.path.isdir(image_path):
    os.mkdir(image_path)
fw_group = fw.lookup( FW_GROUP )
fw_project = fw_group.projects.find_first('label={}'.format(FW_PROJECT))
fw_subjects = fw_project.subjects.iter()

In [5]:
def gunzip(source_filepath, dest_filepath, filename, subject, block_size=65536):
    attempts = 0
    while attempts <= 5:
        try:
            with gzip.open(source_filepath, 'rb') as s_file, open(dest_filepath, 'wb') as d_file:
                while True:
                    block = s_file.read(block_size)
                    if not block:
                        break
                    else:
                        d_file.write(block)
            break
        except Exception as e:
            print("Found an error at ", filename, " with subject ", subject)
            print(e)
            attempts += 1 

In [42]:
count = 0
for fw_subject in fw_subjects:
    count = count + 1
print(count)

73


In [59]:
# Extract all the flywheel data

for fw_subject in fw_subjects:
    if not fw_subject.label.isnumeric():
        continue
    subject_path = os.path.join(image_path, fw_subject.label)
    if not os.path.isdir(subject_path):
        os.mkdir(subject_path)
    for sess in fw_subject.sessions.iter():
        session_path = os.path.join(subject_path, sess.label)
        if not os.path.isdir(session_path):
            os.mkdir(session_path)
        for acq in sess.acquisitions.iter():
            if (acq.label=='Extracted ROIs'):
                continue
            acq_path = os.path.join(session_path, acq.label)
            for f in acq.files:
                if f.type=='dicom':
                    filepath = root + 'flywheel/{}_{}'.format(fw_subject.label,f.name)
                    # if downloaded files don't exist, then unzip them
                    if not os.path.exists(filepath):
                        acq.download_file(f.name, filepath)
                        z = zipfile.ZipFile(filepath)
                        z.extractall(acq_path)
                if f.type=='nifti':
                    nifti_path = os.path.join(session_path, f.name)
                    if not os.path.exists(nifti_path):
                        acq.download_file(f.name, nifti_path)
                        print('Successfully downloaded file', nifti_path)

#                     filepath = root + 'flywheel/Nifti_{}_{}'.format(fw_subject.label,f.name)
#                     # if downloaded files don't exist, then unzip them
#                     if not os.path.exists(filepath):
#                         acq.download_file(f.name, filepath)
#                         print('Successfully downloaded file' + f.name)
#                     nifti_path = os.path.join(session_path, f.name[:-3])
#                     if not os.path.exists(nifti_path):
#                         gunzip(filepath, nifti_path, f.name, fw_subject.label)
    fw_subject = fw_subject.reload()

Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/Young_Raymond_F.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/Ax_PERFUSION_10_t71917.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 13_48_17/SCOUT_1_e1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 13_48_17/SCOUT_1_e1a.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 13_48_17/CO_ST_400.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 13_48_17/SA_ST_401.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 13_

Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/flow_SVDbc_(color)_1001.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/Tmax_(color)_1004.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/rCBV_1012.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/rCBV_(color)_1002.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/FMT_(color)_1003.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/MTT(CVP)_(color)_1005.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_

Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1874966/2019-11-04 13_49_49/CO_BODY_602.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1874966/2019-11-04 13_49_49/Smart_Prep_Series_201.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1874966/2019-11-04 13_49_49/ST_W_C_304_e1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1874966/2019-11-04 13_49_49/ST_W_C_304_e1a.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1874966/2019-11-04 13_49_49/ST_W_C_304_e1a_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1874966/2019-11-04 13_49_49/ST_W_C_304_e1b.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/187496

Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1i.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1i_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1j.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1j_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1k.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1k_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/ST_2_e1l.nii.g

Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/REPEAT_ST_5_e1i_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/REPEAT_ST_5_e1j.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/REPEAT_ST_5_e1k.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/REPEAT_ST_5_e1k_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/REPEAT_ST_5_e1l.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/475328/2020-07-16 10_25_25/REPEAT_ST_5_e1l_Eq_1.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data

Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3260643/2021-04-28 09_26_24/+C_Sag_T2_FLAIR_Straight_15.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3260643/2021-04-28 09_26_24/[NoCorr]_Ax_DTI_Straight_(b800_40_Direc_Apr_28_2021_10-19-40_CDT_1250.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3260643/2021-04-28 09_26_24/[NoCorr]_Ax_DTI_Straight_(b800_40_Direc_Apr_28_2021_10-19-41_CDT_1250.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3260643/2021-04-28 09_26_24/[NoCorr]_Ax_DTI_Straight_(b800_40_Direc_Apr_28_2021_10-19-42_CDT_1250.nii.gz
Successfully downloaded file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3260643/2021-04-28 09_26_24/[NoCorr]_Ax_DTI_Straight_(b800_40_Direc_Apr_28_2021_10-19-43_CDT_1250.nii.gz
Successfully downloaded fil

72


In [32]:
def get_subject(project, label, update=True, **kwargs):
    """Get the Subject container if it exists, else create a new Subject container.
    
    Args:
        project (flywheel.Project): A Flywheel Project.
        label (str): The subject label.
        update (bool): If true, update container with key/value passed as kwargs.
        kwargs (dict): Any key/value properties of subject you would like to update.

    Returns:
        (flywheel.Subject): A Flywheel Subject container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    subject = project.subjects.find_first('label="{}"'.format(label))
    if not subject:
        raise ValueError(f'subject {label} is not found')
        
    if update and kwargs:
        subject.update(**kwargs)

    if subject:
        subject = subject.reload()

    return subject

In [33]:
def get_session(subject, label, update=True, **kwargs):
    """Get the Session container if it exists, else create a new Session container.
    
    Args:
        subject (flywheel.Subject): A Flywheel Subject.
        label (str): The session label.
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Session you would like to update.

    Returns:
        (flywheel.Session): A flywheel Session container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    session = subject.sessions.find_first('label="{}"'.format(label))
    if not session:
        raise ValueError(f'session {label} is not found')
        
    if update and kwargs:
        session.update(**kwargs)

    if session:
        session = session.reload()

    return session

In [34]:
def get_or_create_acquisition(session, label, update=True, **kwargs):
    """Get the Acquisition container if it exists, else create a new Acquisition container.
    
    Args:
        session (flywheel.Session): A Flywheel Session.
        label (str): The Acquisition label.
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.

    Returns:
        (flywheel.Acquisition): A Flywheel Acquisition container.
    """
    
    if not label:
        raise ValueError(f'label is required (currently {label})')
        
    acquisition = session.acquisitions.find_first('label="{}"'.format(label))
    if not acquisition:
        acquisition = session.add_acquisition(label=label)
        
    if update and kwargs:
        acquisition.update(**kwargs)

    if acquisition:
        acquisition = acquisition.reload()

    return acquisition

In [35]:
def upload_file_to_acquisition(acquisition, fp, update=True, **kwargs):
    """Upload file to Acquisition container and update info if `update=True`
    
    Args:
        acquisition (flywheel.Acquisition): A Flywheel Acquisition
        fp (Path-like): Path to file to upload
        update (bool): If true, update container with key/value passed as kwargs.        
        kwargs (dict): Any key/value properties of Acquisition you would like to update.        
    """
    basename = os.path.basename(fp)
    if not os.path.isfile(fp):
        raise ValueError(f'{fp} is not file.')
    attempts = 0
    
    while attempts < 5:
        try: 
            acquisition = acquisition.reload()
            if acquisition.get_file(basename):
                print('file {} already exists'.format(fp))
                return
            else:
                print('uploading', fp)
                acquisition.upload_file(fp)
#                 while not acquistion.get_file(basename):   # to make sure the file is available before performing an update
#                     acquistion = acquistion.reload()
#                     time.sleep(1)
            break
        except Exception as e:
            print('Error exception caught!')
            print(e)
            attempts += 1
            
    if update and kwargs:
        f = acquisition.get_file(basename)
        f.update(**kwargs)

In [11]:
nifti_path = root + 'Nifti_Data'

In [24]:
def changeFileName(file):
    number, name = file.split("-")
    name = name.strip()
    name = name.replace(" ", "_")
    name = name + "_" + number.strip()
    return name

In [93]:
# read in excel spreadsheet of patients documented with all three registration files
df = pandas.read_excel('/Users/cxl037/PycharmProjects/pythonProject1/CompleteRegistration.xlsx', index_col = 0)

# create a dictionary with patient and corresponding registration files
allRegfiles = {}
for _, row in df.iterrows():
    key = str(row.name) + '_' + str(row['MRI_Date'])
    key = key[:-9]
    if key == '1150651_2021-12-17':
        allRegfiles[key] = []
        allRegfiles[key].append('MR.nii')
        allRegfiles[key].append('MR.nii')
        allRegfiles[key].append('MR.nii')
        continue
    allRegfiles[key] = []
    allRegfiles[key].append(changeFileName(row['"+c Ax T1 Stealth bravo"']))
    allRegfiles[key].append(changeFileName(row['"+c COR T1 CUBE VASC"']))
    allRegfiles[key].append(changeFileName(row['"+c Sag CUBE T2 FLAIR"']))

In [ ]:
# iterate through each subject
for subject in os.listdir(image_path):
    if subject != '.DS_Store':
        subject_path = os.path.join(image_path, subject)
    # iterate through each session in the subject
    for session in os.listdir(subject_path):
        if session != '.DS_Store':
            session_path = os.path.join(subject_path, session)
        
        # Intialize the Dicom_reader and walk it through the session folders
        Dicom_reader = DicomReaderWriter(description='RTStructExtractor', arg_max=True, verbose = False)
        Dicom_reader.walk_through_folders(session_path)
        all_rois = Dicom_reader.return_rois(print_rois=True)  # Return a list of all rois present, and print them
        
        # if there are no ROIs, then skip
        if not all_rois:
            print("Skip patient {} because no rois".format(subject))
            continue
        
        # otherwise create a new acquisition
        fw_subject = get_subject(fw_project, subject)
        fw_session = get_session(fw_subject, session)
        fw_acquisition = get_or_create_acquisition(fw_session, 'Extracted ROIs')
        
        # parse through subjects with all the Registration files
        if int(subject) in allRegfiles.keys():
            T1StealthBravo = ''
            T1CubeVasc = ''
            T2CubeFlair = ''
            # go through all the files in the session and if the file is a registration file, then initialize it correspondingly
            for file in os.listdir(session_path):
                regfiles = allRegfiles[int(subject)]
                if regfiles[0] in file:
                    T1StealthBravo = os.path.join(session_path, file)
                if regfiles[1] in file:
                    T1CubeVasc = os.path.join(session_path, file)
                if regfiles[2] in file:
                    T2CubeFlair = os.path.join(session_path, file)
            
            # all of the files were found and are not empty
            if T1StealthBravo and T1CubeVasc and T2CubeFlair:
                # read files into antspy
                fixed = ants.image_read(T1StealthBravo)
                moving = ants.image_read(T1CubeVasc)
                moving2 = ants.image_read(T2CubeFlair)

                # register T1CubeVasc to T1StealthBravo
                regT1CubeVasc = ants.registration(fixed=fixed, moving=moving, type_of_transform='Similarity')

                # register T2CubeFlair to T1StealthBravo
                regT2CubeFlair = ants.registration(fixed=fixed, moving=moving2, type_of_transform='Similarity')

                # output path to write registered files to
                output_path = os.path.join(nifti_path, '{}_T1CubeVasc.nii'.format(subject))
                output_path2 = os.path.join(nifti_path, '{}_T2CubeFlair.nii'.format(subject))

                # write the registered files with forward transform to output path
                ants.image_write(regT1CubeVasc['warpedmovout'], output_path)
                ants.image_write(regT2CubeFlair['warpedmovout'], output_path2)
                
                # upload files to the newly created acquisition
                upload_file_to_acquisition(fw_acquisition, output_path)
                upload_file_to_acquisition(fw_acquisition, output_path2)
        
        # for each of the rois, create a nifti file for the extracted roi
        for roi in all_rois:
            # initialize the contour_names and associations
            # Associations work as {'variant_name': 'desired_name'}
            Contour_Names = [roi]
            associations = {roi:roi}
            Dicom_reader.set_contour_names_and_associations(Contour_Names=Contour_Names, associations=associations)
            
            # get the images and mask which have this roi
            indexes = Dicom_reader.which_indexes_have_all_rois()
            pt_indx = indexes[0]
            Dicom_reader.set_index(pt_indx)
            Dicom_reader.get_images_and_mask()
            dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
            mask_sitk_handle = Dicom_reader.annotation_handle # SimpleITK mask handle
            
            # Get the base image for the mask
            base_image = Dicom_reader.series_instances_dictionary[pt_indx]['Description']

            # write the image to the output nifti path
            image_nifti_path = os.path.join(nifti_path, '{}_{}.nii'.format(subject, base_image))
            mask_nifti_path = os.path.join(nifti_path, '{}_{}_Mask.nii'.format(subject, roi))
            sitk.WriteImage(dicom_sitk_handle, image_nifti_path)
            sitk.WriteImage(mask_sitk_handle, mask_nifti_path)

            # upload extracted rois to the newly created acquisition
            upload_file_to_acquisition(fw_acquisition, image_nifti_path)
            upload_file_to_acquisition(fw_acquisition, mask_nifti_path)
    

Loading through DICOM files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:01<00:00, 12.80it/s]


Index 0, description 3pl loc FGRE at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495608/2011-12-14 07_11_56/1 - 3pl loc FGRE
Index 1, description Sag T1 FLAIR at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495608/2011-12-14 07_11_56/2 - Sag T1 FLAIR
Index 2, description Ax DWI at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495608/2011-12-14 07_11_56/4 - Ax DWI
Index 3, description Ax T2 fat sat  at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495608/2011-12-14 07_11_56/5 - Ax T2 fat sat
Index 4, description Ax T1 BRAVO  at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495608/2011-12-14 07_11_56/6 - Ax T1 BRAVO
Index 5, description +C Sag CUBE T2 FLAIR fat at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495608/2011-12-14 07_11_56/7 - +C Sag CUBE T2 FLAIR fat
Index 6, description +C Ax T1 Stealth BRAVO at /Users/cxl037/PycharmProjects/pyt

Loading through DICOM files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  9.91it/s]


The following ROIs were found
bone_igrt
brain
brain-gtvs
brainstem
external
globe_l
globe_r
otv
gtv
ptv
The following indexes have all ROIs present
Index 0, located at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/0894326/2017-07-17 09_23_59/4 - SAFIRE 2 STEREO_HEAD_RAD_PLAN/Jegerlehner^Marilyn^J/CT/20170717/4
Finished listing present indexes
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/0894326_SAFIRE 2 STEREO_HEAD_RAD_PLAN .nii already exists
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/0894326_bone_igrt_Mask.nii already exists
The following indexes have all ROIs present
Index 0, located at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/0894326/2017-07-17 09_23_59/4 - SAFIRE 2 STEREO_HEAD_RAD_PLAN/Jegerlehner^Marilyn^J/CT/20170717/4
Finished listing present indexes
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/0894326_SAFIRE 2 STEREO_HEAD_RAD_PLAN .nii alr

Loading through DICOM files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  9.55it/s]


The following ROIs were found
gtv1
ptv1
The following indexes have all ROIs present
Index 0, located at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2018-03-15 08_54_50/4 - SAFIRE 2 STEREO_HEAD_RAD_PLAN/Turner^Nelson/CT/20180315/4
Finished listing present indexes
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/1766382_SAFIRE 2 STEREO_HEAD_RAD_PLAN .nii already exists
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/1766382_gtv1_Mask.nii already exists
The following indexes have all ROIs present
Index 0, located at /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2018-03-15 08_54_50/4 - SAFIRE 2 STEREO_HEAD_RAD_PLAN/Turner^Nelson/CT/20180315/4
Finished listing present indexes
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data/1766382_SAFIRE 2 STEREO_HEAD_RAD_PLAN .nii already exists
file /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Nifti_Data

Loading through DICOM files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00,  9.77it/s]

In [54]:
count = 0
for subject in os.listdir(image_path):
    count = count + 1
print(count)

73


In [95]:
count = 0
allSubjects = [] 
for subject in os.listdir(image_path):
    if subject == '.DS_Store':
        continue
    subject_path = os.path.join(image_path, subject)
    for session in os.listdir(subject_path):
        if session == '.DS_Store':
            continue
        session_path = os.path.join(subject_path, session)
        fw_subject = get_subject(fw_project, subject)
        fw_session = get_session(fw_subject, session)
        
        key = subject + '_' + session[:10]
        # parse through subjects with all the Registration files
        if key in allRegfiles.keys():
            print(key)
            T1StealthBravo = ''
            T1CubeVasc = ''
            T2CubeFlair = ''
            # go through all the files in the session and if the file is a registration file, then initialize it correspondingly
            for file in os.listdir(session_path):
                regfiles = allRegfiles[key]
                if regfiles[0] in file:
                    T1StealthBravo = os.path.join(session_path, file)
                if regfiles[1] in file:
                    T1CubeVasc = os.path.join(session_path, file)
                if regfiles[2] in file:
                    T2CubeFlair = os.path.join(session_path, file)
            # all of the files were found and are not empty
            if T1StealthBravo and T1CubeVasc and T2CubeFlair:
                count = count + 1
                allSubjects.append(key)
                print(T1StealthBravo, T1CubeVasc, T2CubeFlair)
                print(subject)
                print(fw_session.label)

1766382_2020-07-23
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/+c_Ax_T1_Stealth_Bravo_10.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/+c_COR_T1_CUBE_VASC_11.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1766382/2020-07-23 11_37_42/+c_Sag_T2_FLAIR_Cube_9.nii.gz
1766382
2020-07-23 11_37_42
2495206_2017-11-03
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495206/2017-11-03 12_59_28/+Ax_3dT1_BRAVO_Stealth_11.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495206/2017-11-03 12_59_28/+Cor_T1_CUBE_VASC_12.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2495206/2017-11-03 12_59_28/+Sag_CUBE_FLAIR_10.nii.gz
2495206
2017-11-03 12_59_28
3087972_2017-12-20
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3087972/2017-12-20 13_31_47/+C_Ax_T1_BRAVO_Stealth_14.ni

3056129_2017-07-22
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3056129/2017-07-22 13_25_20/+c_AxT1_Stealth_BRAVO_9.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3056129/2017-07-22 13_25_20/+c_COR_T1_CUBE_VASC_10.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3056129/2017-07-22 13_25_20/+c_T2_CUBE_FLAIR_Sag_8.nii.gz
3056129
2017-07-22 13_25_20
1244948_2020-09-15
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1244948/2020-09-15 13_20_04/+c_Ax_T1_Stealth_bravo_10.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1244948/2020-09-15 13_20_04/+c_COR_CUBE_T1_VASC_11.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1244948/2020-09-15 13_20_04/+c_Sag_CUBE_T2_FLAIR_9.nii.gz
1244948
2020-09-15 13_20_04
2489655_2017-09-05
2489655_2017-09-05
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2489655/2017-09-05 19_24_24/+C_Ax

2456301_2021-05-21
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2456301/2021-05-21 16_58_26/POST_AX_3D_T1_BRAVO_1.2MM_13.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2456301/2021-05-21 16_58_26/POST_COR_CUBE_T1_VASC_10.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/2456301/2021-05-21 16_58_26/POST_SAG_T2_FS_CUBE_FLAIR_12.nii.gz
2456301
2021-05-21 16_58_26
1759492_2017-04-04
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1759492/2017-04-04 17_20_59/POST_AX_3D_T1_BRAVO_10.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1759492/2017-04-04 17_20_59/POST_Cor_T1_Cube_VASC_11.nii.gz /Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/1759492/2017-04-04 17_20_59/POST_SAG_T2_FS_CUBE_FLAIR_9.nii.gz
1759492
2017-04-04 17_20_59
3303688_2021-03-16
/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/Image_Data/3303688/2021-03-16 10_09_18/3D_Ax_T

In [96]:
print(count)

48


In [90]:
actualList = allRegfiles.keys()
print(len(actualList))
print(allSubjects)
print(len(allSubjects))
print(list(actualList))
print(set(actualList) - set(allSubjects))

48
['1766382_2020-07-23', '2495206_2017-11-03', '3087972_2017-12-20', '3305568_2019-03-19', '2792388_2018-06-15', '392811_2019-02-15', '1935437_2019-01-25', '1935437_2018-11-01', '1577489_2021-06-22', '1108503_2021-09-27', '1108503_2020-12-14', '894326_2018-05-09', '1649749_2016-12-30', '1649749_2018-07-16', '3039706_2019-09-16', '3188513_2019-07-12', '2851803_2021-10-11', '1045531_2021-01-30', '3056129_2017-07-22', '1244948_2020-09-15', '2489655_2017-09-05', '3294322_2019-02-11', '1455583_2018-01-09', '2762223_2019-09-09', '2628744_2015-11-04', '475328_2020-07-16', '3056961_2018-07-11', '3086126_2020-11-30', '2775036_2021-01-06', '1183497_2020-01-02', '1425124_2019-02-12', '2691237_2017-07-18', '1267610_2021-09-03', '1874966_2019-11-04', '1511647_2021-07-30', '2456301_2021-05-21', '1759492_2017-04-04', '3303688_2021-03-16', '2849119_2019-03-20', '1852832_2015-10-13', '2452082_2020-08-14', '2938340_2018-10-03', '1828569_2019-02-04', '1833770_2021-10-05', '2890799_2017-09-26']
45
['3928